In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
from skimage.draw import polygon
from tqdm import tqdm

def parse_volume_tracings(volume_tracings_csv):
    assert os.path.exists(volume_tracings_csv), f"File not found: {volume_tracings_csv}"

    try:
        data = pd.read_csv(volume_tracings_csv, encoding="utf-8-sig", engine="python")
        print("CSV loaded successfully!")
    except Exception as e:
        raise ValueError(f"Failed to read CSV file: {e}")

    expected_columns = ["FileName", "X1", "Y1", "X2", "Y2", "Frame"]
    if not all(col in data.columns for col in expected_columns):
        raise ValueError(f"CSV columns do not match expected format. Found columns: {data.columns}")

    frames = {}
    trace = {}

    for _, row in data.iterrows():
        filename, x1, y1, x2, y2, frame = row
        x1, y1, x2, y2 = map(float, [x1, y1, x2, y2])
        frame = int(frame)

        if filename not in frames:
            frames[filename] = []
            trace[filename] = {}

        if frame not in trace[filename]:
            frames[filename].append(frame)
            trace[filename][frame] = []

        trace[filename][frame].append((x1, y1, x2, y2))

    return frames, trace

def generate_mask(trace_data, frame_shape):
    points = np.array(trace_data)
    x1, y1 = points[:, 0], points[:, 1]
    x2, y2 = points[:, 2], points[:, 3]

    x = np.concatenate((x1[1:], np.flip(x2[1:])))
    y = np.concatenate((y1[1:], np.flip(y2[1:])))

    r, c = polygon(np.rint(y).astype(int), np.rint(x).astype(int), frame_shape)
    mask = np.zeros(frame_shape, dtype=np.uint8)
    mask[r, c] = 255

    return mask

def extract_frames_and_masks(video_dir, output_dir, frames, trace):
    os.makedirs(output_dir, exist_ok=True)
    frame_output_dir = os.path.join(output_dir, "frames")
    mask_output_dir = os.path.join(output_dir, "masks")
    log_file_path = os.path.join(output_dir, "log.txt")
    os.makedirs(frame_output_dir, exist_ok=True)
    os.makedirs(mask_output_dir, exist_ok=True)

    with open(log_file_path, "w") as log_file:
        for video_name in tqdm(frames.keys(), desc="Processing videos"):
            video_path = os.path.join(video_dir, video_name)
            if not os.path.exists(video_path):
                error_message = f"Video not found: {video_path}\n"
                print(error_message.strip())
                log_file.write(error_message)
                continue

            cap = cv2.VideoCapture(video_path)
            if not cap.isOpened():
                error_message = f"Failed to open video file: {video_path}\n"
                print(error_message.strip())
                log_file.write(error_message)
                continue

            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            log_file.write(f"Processing {video_name}: {total_frames} frames, resolution {width}x{height}\n")

            for frame_idx in frames[video_name]:
                if frame_idx >= total_frames:
                    error_message = f"Frame index {frame_idx} out of range for video {video_name}\n"
                    print(error_message.strip())
                    log_file.write(error_message)
                    continue

                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
                ret, frame = cap.read()
                if not ret or frame is None:
                    error_message = f"Failed to read frame {frame_idx} from {video_name}\n"
                    print(error_message.strip())
                    log_file.write(error_message)
                    continue

                frame_filename = f"{os.path.splitext(video_name)[0]}_frame{frame_idx:03d}.png"
                frame_path = os.path.join(frame_output_dir, frame_filename)
                mask_path = os.path.join(mask_output_dir, frame_filename)

                # Save frame
                if not cv2.imwrite(frame_path, frame):
                    error_message = f"Failed to save frame {frame_filename} to {frame_output_dir}\n"
                    print(error_message.strip())
                    log_file.write(error_message)
                    continue

                # Generate and save mask
                try:
                    mask = generate_mask(trace[video_name][frame_idx], (height, width))
                    if not cv2.imwrite(mask_path, mask):
                        error_message = f"Failed to save mask {frame_filename} to {mask_output_dir}\n"
                        print(error_message.strip())
                        log_file.write(error_message)
                except Exception as e:
                    error_message = f"Error generating/saving mask for frame {frame_idx} in {video_name}: {e}\n"
                    print(error_message.strip())
                    log_file.write(error_message)

            cap.release()

if __name__ == "__main__":
    video_dir = "dynamic\\a4c-video-dir\\Videos"
    volume_tracings_csv = "dynamic\\a4c-video-dir\\VolumeTracings.csv"
    output_dir = "dynamic\\mask_and_label_frame"

    frames, trace = parse_volume_tracings(volume_tracings_csv)
    extract_frames_and_masks(video_dir, output_dir, frames, trace)
    print("Extraction complete. Logs saved to log.txt.")
